
# 01b_clean_reference_and_customer.ipynb
**Auxiliary Data Cleaning — SKU Reference & Customer Repeat Summary**  
Author: **Derrick Wong**

This notebook cleans two auxiliary tables:
- `fmcg_sku_reference.csv` → product master (SKU, Product_Category, Unit_Price, etc.)
- `fmcg_customer_repeat_summary.csv` → per-customer repeat stats (Customer_ID, Orders, Repeat_Frequency, etc.)

**What this does**
1. Load files (from your project `/data` folder)  
2. Profile: head/info/describe + missing counts  
3. Clean: trim strings, standardize case, fix types  
4. Handle missing values (mode/median), remove duplicates  
5. Harmonize category/channel/territory labels where present  
6. Save cleaned outputs into `/clean/`  
7. Print before/after shapes and key stats


## 1) Setup paths & load data

In [1]:

import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 100)

# ---- Set your project base path (fixed to Derrick's machine) ----
FORCED_BASE = Path(r"C:\Users\inchr\Downloads\Capstone Associate Data Analyst\omnichannel-growth-engine")
BASE_DIR = FORCED_BASE if FORCED_BASE.exists() else Path.cwd().parent

DATA_DIR  = BASE_DIR / "data"
CLEAN_DIR = BASE_DIR / "clean"
CLEAN_DIR.mkdir(exist_ok=True, parents=True)

SKU_PATH   = DATA_DIR / "fmcg_sku_reference.csv"
CUST_PATH  = DATA_DIR / "fmcg_customer_repeat_summary.csv"

print("📁 BASE_DIR :", BASE_DIR)
print("📁 DATA_DIR :", DATA_DIR)
print("📁 CLEAN_DIR:", CLEAN_DIR)
print("🔎 SKU file exists?  ", SKU_PATH.exists(), "→", SKU_PATH)
print("🔎 CUST file exists? ", CUST_PATH.exists(), "→", CUST_PATH)

# Fallback to uploaded files (from this chat session) if not found locally
fallback_sku  = Path('/mnt/data/fmcg_sku_reference.csv')
fallback_cust = Path('/mnt/data/fmcg_customer_repeat_summary.csv')
if not SKU_PATH.exists() and fallback_sku.exists():
    SKU_PATH = fallback_sku
if not CUST_PATH.exists() and fallback_cust.exists():
    CUST_PATH = fallback_cust

def load_df(path):
    if not path.exists():
        print(f"⚠️ File not found: {path}")
        return None
    try:
        return pd.read_csv(path, low_memory=False)
    except Exception as e:
        print(f"⚠️ Could not read {path} as CSV: {e}")
        return None

sku  = load_df(SKU_PATH)
cust = load_df(CUST_PATH)

print("✅ Loaded tables:",
      {"sku": None if sku is None else sku.shape,
       "cust": None if cust is None else cust.shape})


📁 BASE_DIR : C:\Users\inchr\Downloads\Capstone Associate Data Analyst\omnichannel-growth-engine
📁 DATA_DIR : C:\Users\inchr\Downloads\Capstone Associate Data Analyst\omnichannel-growth-engine\data
📁 CLEAN_DIR: C:\Users\inchr\Downloads\Capstone Associate Data Analyst\omnichannel-growth-engine\clean
🔎 SKU file exists?   True → C:\Users\inchr\Downloads\Capstone Associate Data Analyst\omnichannel-growth-engine\data\fmcg_sku_reference.csv
🔎 CUST file exists?  True → C:\Users\inchr\Downloads\Capstone Associate Data Analyst\omnichannel-growth-engine\data\fmcg_customer_repeat_summary.csv
✅ Loaded tables: {'sku': (18, 4), 'cust': (2200, 5)}


## 2) Helper functions

In [2]:

def log_profile(name, df):
    print(f"\n=== {name}: HEAD ===")
    display(df.head(3))
    print(f"\n=== {name}: INFO ===")
    display(df.info())
    print(f"\n=== {name}: DESCRIBE (numeric) ===")
    display(df.describe(include=[np.number]))
    print(f"\n=== {name}: DESCRIBE (object) ===")
    display(df.describe(include=['object']))
    print(f"\n=== {name}: NULL COUNTS ===")
    display(df.isna().sum().sort_values(ascending=False))

def trim_and_case(df, obj_cols, title_case=False):
    for c in obj_cols:
        df[c] = df[c].astype(str).str.strip()
        if title_case:
            df[c] = df[c].str.title()
    return df

def fill_missing(df, strategy_map):
    for col, strat in strategy_map.items():
        if col not in df.columns:
            continue
        if strat == "mode":
            mode_val = df[col].mode(dropna=True)
            if not mode_val.empty:
                df[col] = df[col].fillna(mode_val.iloc[0])
        elif strat == "median":
            df[col] = df[col].fillna(df[col].median())
        elif strat == "zero":
            df[col] = df[col].fillna(0)
        elif isinstance(strat, (int,float,str)):
            df[col] = df[col].fillna(strat)
    return df

def dedupe(df, subset_cols=None):
    before = df.shape[0]
    df = df.drop_duplicates(subset=subset_cols).copy()
    after = df.shape[0]
    print(f"🧹 Duplicates removed: {before - after} (from {before} → {after})")
    return df

def harmonize_values(series, mapping):
    return series.replace(mapping).astype(str).str.strip()


## 3) Clean: SKU Reference (`fmcg_sku_reference.csv`)

In [3]:

if sku is not None:
    print("📦 SKU Reference — BEFORE:", sku.shape)
    log_profile("SKU", sku)

    # Identify likely columns
    cols = {c.lower(): c for c in sku.columns}
    sku_id   = cols.get("sku_id") or cols.get("skuid") or cols.get("sku")
    cat_col  = cols.get("product_category") or cols.get("category") or cols.get("sku_category")
    name_col = cols.get("product_name") or cols.get("name")
    brand    = cols.get("brand")
    price    = cols.get("unit_price") or cols.get("unitprice") or cols.get("price")
    size     = cols.get("pack_size") or cols.get("packsize") or cols.get("size")

    # Basic cleaning
    obj_cols = [c for c in [sku_id, cat_col, name_col, brand, size] if c and c in sku.columns]
    sku = trim_and_case(sku, obj_cols, title_case=True)

    # Types
    if price and price in sku.columns:
        sku[price] = pd.to_numeric(sku[price], errors="coerce")

    # Fill missing
    sku = fill_missing(sku, {
        brand: "mode",
        cat_col: "mode",
        price: "median"
    })

    # Remove duplicates by SKU_ID if present
    if sku_id and sku_id in sku.columns:
        sku = dedupe(sku, subset_cols=[sku_id])
    else:
        sku = dedupe(sku)

    print("\n📦 SKU Reference — AFTER:", sku.shape)
    log_profile("SKU (CLEANED)", sku)

    # Save
    out_sku = CLEAN_DIR / "fmcg_sku_reference_clean.csv"
    sku.to_csv(out_sku, index=False)
    print("✅ Saved:", out_sku.resolve())
else:
    print("⚠️ SKU file missing; skipping.")


📦 SKU Reference — BEFORE: (18, 4)

=== SKU: HEAD ===


,SKU_ID,Category,BasePrice,BaseDemand
0,INS01,Instant Noodles,1.15,185
1,INS02,Instant Noodles,1.42,183
2,INS03,Instant Noodles,1.29,228



=== SKU: INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SKU_ID      18 non-null     object 
 1   Category    18 non-null     object 
 2   BasePrice   18 non-null     float64
 3   BaseDemand  18 non-null     int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 708.0+ bytes


None


=== SKU: DESCRIBE (numeric) ===


...


=== SKU (CLEANED): DESCRIBE (object) ===


,SKU_ID,Category
count,18,18
unique,18,4
top,Ins01,Instant Noodles
freq,1,5



=== SKU (CLEANED): NULL COUNTS ===


SKU_ID        0
Category      0
BasePrice     0
BaseDemand    0
dtype: int64

✅ Saved: C:\Users\inchr\Downloads\Capstone Associate Data Analyst\omnichannel-growth-engine\clean\fmcg_sku_reference_clean.csv


## 4) Clean: Customer Repeat Summary (`fmcg_customer_repeat_summary.csv`)

In [4]:

if cust is not None:
    print("👥 Customer Repeat — BEFORE:", cust.shape)
    log_profile("CUSTOMER", cust)

    # Likely columns
    cols = {c.lower(): c for c in cust.columns}
    cust_id  = cols.get("customer_id") or cols.get("cust_id") or cols.get("customerid")
    orders   = cols.get("orders") or cols.get("order_count") or cols.get("orders_count")
    revenue  = cols.get("revenue") or cols.get("total_revenue") or cols.get("spend")
    repeat_f = cols.get("repeat_frequency") or cols.get("repeatfreq") or cols.get("frequency")
    last_dt  = cols.get("last_order_date") or cols.get("last_date") or cols.get("last_purchase")

    # Clean strings
    obj_cols = [c for c in [cust_id] if c and c in cust.columns]
    cust = trim_and_case(cust, obj_cols, title_case=False)

    # Types
    if orders and orders in cust.columns:
        cust[orders] = pd.to_numeric(cust[orders], errors="coerce")
    if revenue and revenue in cust.columns:
        cust[revenue] = pd.to_numeric(cust[revenue], errors="coerce")
    if repeat_f and repeat_f in cust.columns:
        cust[repeat_f] = pd.to_numeric(cust[repeat_f], errors="coerce")
    if last_dt and last_dt in cust.columns:
        cust[last_dt] = pd.to_datetime(cust[last_dt], errors="coerce")

    # Fill missing
    cust = fill_missing(cust, {
        orders: "median",
        revenue: "median",
        repeat_f: "median"
    })

    # Remove duplicates by Customer_ID if present
    if cust_id and cust_id in cust.columns:
        cust = dedupe(cust, subset_cols=[cust_id])
    else:
        cust = dedupe(cust)

    # Derived metrics
    if orders and orders in cust.columns and revenue and revenue in cust.columns:
        cust["AOV"] = cust[revenue] / cust[orders].replace(0, np.nan)

    print("\n👥 Customer Repeat — AFTER:", cust.shape)
    log_profile("CUSTOMER (CLEANED)", cust)

    # Save
    out_cust = CLEAN_DIR / "fmcg_customer_repeat_summary_clean.csv"
    cust.to_csv(out_cust, index=False)
    print("✅ Saved:", out_cust.resolve())
else:
    print("⚠️ Customer file missing; skipping.")


👥 Customer Repeat — BEFORE: (2200, 5)

=== CUSTOMER: HEAD ===


,Customer_ID,Orders_6m,Total_Units_6m,Total_Revenue_6m,Channels_Used
0,C00039370,56,422,581.58,4
1,C00292696,102,684,1123.94,4
2,C00325576,37,184,276.63,4



=== CUSTOMER: INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Customer_ID       2200 non-null   object 
 1   Orders_6m         2200 non-null   int64  
 2   Total_Units_6m    2200 non-null   int64  
 3   Total_Revenue_6m  2200 non-null   float64
 4   Channels_Used     2200 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB


None


=== CUSTOMER: DESCRIBE (numeric) ===


...


=== CUSTOMER (CLEANED): DESCRIBE (object) ===


,Customer_ID
count,2200
unique,2200
top,C99995818
freq,1



=== CUSTOMER (CLEANED): NULL COUNTS ===


Customer_ID         0
Orders_6m           0
Total_Units_6m      0
Total_Revenue_6m    0
Channels_Used       0
dtype: int64

✅ Saved: C:\Users\inchr\Downloads\Capstone Associate Data Analyst\omnichannel-growth-engine\clean\fmcg_customer_repeat_summary_clean.csv


## 5) Next steps

In [5]:

print("🎯 Cleaning finished. Next:")
print("1) Ensure 'cleaned_fmcg_omnichannel_sales.csv' (fact) is ready in /clean")
print("2) Load three tables in Power BI:")
print("   - Fact: cleaned_fmcg_omnichannel_sales.csv")
print("   - Dim:  fmcg_sku_reference_clean.csv")
print("   - Dim:  fmcg_customer_repeat_summary_clean.csv")
print("3) Create relationships by SKU_ID and Customer_ID as applicable.")


🎯 Cleaning finished. Next:
1) Ensure 'cleaned_fmcg_omnichannel_sales.csv' (fact) is ready in /clean
2) Load three tables in Power BI:
   - Fact: cleaned_fmcg_omnichannel_sales.csv
   - Dim:  fmcg_sku_reference_clean.csv
   - Dim:  fmcg_customer_repeat_summary_clean.csv
3) Create relationships by SKU_ID and Customer_ID as applicable.
